In [1]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.decomposition import FastICA
from sklearn import random_projection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import homogeneity_score
from collections import Counter
import pandas as pd
import numpy as np
import random
from pytictoc import TicToc
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
t = TicToc()

In [2]:
dfTraining = pd.read_csv('trainingData_treated.csv')
dfTest = pd.read_csv('testData_treated.csv')

In [3]:
# Preparing the dataset for clustering and later for the neural network

dfTest = dfTest.groupby('adjusted_gross_income', group_keys=False).apply(lambda x: x.sample(2000, random_state=1))
dfTraining = dfTraining.groupby('adjusted_gross_income', group_keys=False).apply(lambda x: x.sample(8000, random_state=1))
dfTrainingX = pd.concat((dfTraining["capital_gains"]/max(dfTraining["capital_gains"]),
    dfTraining["divdends_from_stocks"]/max(dfTraining["divdends_from_stocks"]),                    
    pd.get_dummies(dfTraining["education"]),
    pd.get_dummies(dfTraining["class_of_worker"]),
    pd.get_dummies(dfTraining["race"]),
    pd.get_dummies(dfTraining["sex"])), axis=1)

dfTrainingY = pd.get_dummies(dfTraining["adjusted_gross_income"])
dfTrainingY = dfTrainingY.drop([" - 50000."],axis=1)
dfTrainingY = dfTrainingY.rename(columns={" 50000+.": "Income"})
dfTestX = pd.concat((dfTest["capital_gains"]/max(dfTest["capital_gains"]),
    dfTest["divdends_from_stocks"]/max(dfTest["divdends_from_stocks"]),                    
    pd.get_dummies(dfTest["education"]),
    pd.get_dummies(dfTest["class_of_worker"]),
    pd.get_dummies(dfTest["race"]),
    pd.get_dummies(dfTest["sex"])), axis=1)

dfTestY = pd.get_dummies(dfTest["adjusted_gross_income"])
dfTestY = dfTestY.drop([" - 50000."],axis=1)
dfTestY = dfTestY.rename(columns={" 50000+.": "Income"})
# Initialize Neural Network
nnBP = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(2, 2), random_state=1,max_iter=10000,activation='relu')
t.tic()
nnBP.fit(dfTrainingX, dfTrainingY.values.ravel())
taux = t.tocvalue()
dfTestX[" Without pay"]=0
dfTestX = dfTestX.reindex(columns=dfTrainingX.columns.values.tolist())
dfTestY_Pred = nnBP.predict(dfTestX)
dfTrainingY_Pred = nnBP.predict(dfTrainingX)
print(accuracy_score(dfTrainingY, dfTrainingY_Pred))
print(accuracy_score(dfTestY, dfTestY_Pred))
neuralnetwork_sol = pd.DataFrame([['Base',0,0,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])


0.8293125
0.82775


In [4]:
label_real = dfTrainingY['Income'].tolist()
corpus = dfTrainingX
print(corpus.shape)


(16000, 35)


In [5]:
homogeneity_sol = pd.DataFrame([['Test',0,0,0.000,0.000,0.000]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,WCSS,Time
0,Test,0,0,0.0,0.0,0.0


In [6]:
corpus = corpus.fillna(0)
corpus[corpus > 1] = 1
#columnas = list(corpus.columns)
#columnas_words = vocabulary["ID"]=

    

In [7]:
t.tic()
pca = PCA(n_components=35, random_state = 19830526)
pca.fit(corpus)
t.toc('PCA Calibration with 35 components')

PCA Calibration with 35 components 0.132225 seconds.


In [8]:
np.cumsum(pca.explained_variance_ratio_)

array([0.23651296, 0.39733528, 0.48549611, 0.55870874, 0.63017314,
       0.69266135, 0.74314716, 0.77315496, 0.79814576, 0.82232179,
       0.84408099, 0.86303625, 0.87962617, 0.89505644, 0.9090803 ,
       0.92188629, 0.93355204, 0.94343579, 0.95320409, 0.96267595,
       0.97050168, 0.97726495, 0.98330776, 0.9874877 , 0.99149861,
       0.99470947, 0.99713988, 0.99826676, 0.99928197, 0.99978577,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [9]:
sum(pca.explained_variance_ratio_)

1.0000000000000002

In [10]:
t.tic()
corpus_pca = pca.transform(corpus)
t.toc('PCA transform took')

PCA transform took 0.024726 seconds.


In [11]:
# Running PCA First
dfTestXPCA = pca.transform(dfTestX)
dfTrainingXPCA = pca.transform(dfTrainingX)
labels_pca = []
for iteCom in range(1,35):
    t.tic()
    nnBP.fit(dfTrainingXPCA[:,0:iteCom], dfTrainingY.values.ravel())
    taux = t.tocvalue()
    dfTestY_Pred = nnBP.predict(dfTestXPCA[:,0:iteCom])
    dfTrainingY_Pred = nnBP.predict(dfTrainingXPCA[:,0:iteCom])
    print("Neural Netwotk with PCA with "+str(iteCom)+" features took to run " + str(taux))
    new_row = pd.DataFrame([["PCA",iteCom,0,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
    neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
    for ite in range(1,20):
        t.tic()
        kmeans_pca = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_pca.fit(corpus_pca[:,0:iteCom])
        labels_pca.append(kmeans_pca.labels_)
        taux = t.tocvalue()
        print("PCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["PCA",iteCom,ite,homogeneity_score(label_real, kmeans_pca.labels_),kmeans_pca.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])        
        t.tic()
        nnBP.fit(pd.get_dummies(kmeans_pca.labels_), dfTrainingY.values.ravel())
        taux = t.tocvalue()
        dfTestY_Pred = nnBP.predict(pd.get_dummies(kmeans_pca.predict(dfTestXPCA[:,0:iteCom])))
        dfTrainingY_Pred = nnBP.predict(pd.get_dummies(kmeans_pca.labels_))
        print("Neural Netwotk with PCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["PCA",iteCom,ite,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
        neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])

Neural Netwotk with PCA with 1 features took to run 0.784421800000004
PCA with 1 features and 1 clusters took to run 0.5133263000000028
Neural Netwotk with PCA with 1 features and 1 clusters took to run 0.08695620000000304
PCA with 1 features and 2 clusters took to run 0.19521950000000032
Neural Netwotk with PCA with 1 features and 2 clusters took to run 0.06951109999999971
PCA with 1 features and 3 clusters took to run 0.301423199999995
Neural Netwotk with PCA with 1 features and 3 clusters took to run 0.34105249999999643
PCA with 1 features and 4 clusters took to run 0.3473060999999973
Neural Netwotk with PCA with 1 features and 4 clusters took to run 0.3018906000000001
PCA with 1 features and 5 clusters took to run 0.33209839999999957
Neural Netwotk with PCA with 1 features and 5 clusters took to run 0.23543000000000092
PCA with 1 features and 6 clusters took to run 0.6073966000000013
Neural Netwotk with PCA with 1 features and 6 clusters took to run 0.08784390000000286
PCA with 1 f

Neural Netwotk with PCA with 3 features and 15 clusters took to run 0.18864659999999844
PCA with 3 features and 16 clusters took to run 0.16180140000000165
Neural Netwotk with PCA with 3 features and 16 clusters took to run 0.5097709000000066
PCA with 3 features and 17 clusters took to run 0.15867009999999482
Neural Netwotk with PCA with 3 features and 17 clusters took to run 0.1597363999999999
PCA with 3 features and 18 clusters took to run 0.173132300000006
Neural Netwotk with PCA with 3 features and 18 clusters took to run 0.3386308999999983
PCA with 3 features and 19 clusters took to run 0.2069385000000068
Neural Netwotk with PCA with 3 features and 19 clusters took to run 0.03188710000000583
Neural Netwotk with PCA with 4 features took to run 3.9047134000000057
PCA with 4 features and 1 clusters took to run 0.019274599999988595
Neural Netwotk with PCA with 4 features and 1 clusters took to run 0.030645799999987844
PCA with 4 features and 2 clusters took to run 0.0324750999999992
N

Neural Netwotk with PCA with 6 features and 10 clusters took to run 0.14047589999999843
PCA with 6 features and 11 clusters took to run 0.12606329999999843
Neural Netwotk with PCA with 6 features and 11 clusters took to run 0.08536019999999667
PCA with 6 features and 12 clusters took to run 0.1664104999999978
Neural Netwotk with PCA with 6 features and 12 clusters took to run 0.1743412999999947
PCA with 6 features and 13 clusters took to run 0.1748528000000107
Neural Netwotk with PCA with 6 features and 13 clusters took to run 0.10222709999999324
PCA with 6 features and 14 clusters took to run 0.1460600999999997
Neural Netwotk with PCA with 6 features and 14 clusters took to run 0.1863002999999992
PCA with 6 features and 15 clusters took to run 0.15066609999999514
Neural Netwotk with PCA with 6 features and 15 clusters took to run 0.10505739999999264
PCA with 6 features and 16 clusters took to run 0.19337570000000426
Neural Netwotk with PCA with 6 features and 16 clusters took to run 0

Neural Netwotk with PCA with 9 features and 5 clusters took to run 0.10039139999997815
PCA with 9 features and 6 clusters took to run 0.10262300000002256
Neural Netwotk with PCA with 9 features and 6 clusters took to run 0.025903399999975818
PCA with 9 features and 7 clusters took to run 0.11500569999998334
Neural Netwotk with PCA with 9 features and 7 clusters took to run 0.12035870000002546
PCA with 9 features and 8 clusters took to run 0.10171189999999797
Neural Netwotk with PCA with 9 features and 8 clusters took to run 0.08689269999999283
PCA with 9 features and 9 clusters took to run 0.11762120000000209
Neural Netwotk with PCA with 9 features and 9 clusters took to run 0.2354058999999893
PCA with 9 features and 10 clusters took to run 0.12388559999999416
Neural Netwotk with PCA with 9 features and 10 clusters took to run 0.11058840000001169
PCA with 9 features and 11 clusters took to run 0.12498349999998482
Neural Netwotk with PCA with 9 features and 11 clusters took to run 0.111

PCA with 11 features and 19 clusters took to run 0.5814057999999989
Neural Netwotk with PCA with 11 features and 19 clusters took to run 0.032888799999994944
Neural Netwotk with PCA with 12 features took to run 7.743152699999996
PCA with 12 features and 1 clusters took to run 0.02263990000000149
Neural Netwotk with PCA with 12 features and 1 clusters took to run 0.034243000000003576
PCA with 12 features and 2 clusters took to run 0.037214300000016465
Neural Netwotk with PCA with 12 features and 2 clusters took to run 0.03149209999997993
PCA with 12 features and 3 clusters took to run 0.071423899999985
Neural Netwotk with PCA with 12 features and 3 clusters took to run 0.07779240000002119
PCA with 12 features and 4 clusters took to run 0.07585729999999558
Neural Netwotk with PCA with 12 features and 4 clusters took to run 0.0582704999999919
PCA with 12 features and 5 clusters took to run 0.07941480000002343
Neural Netwotk with PCA with 12 features and 5 clusters took to run 0.1418157999

...

Neural Netwotk with PCA with 22 features and 16 clusters took to run 1.3407392999999956
PCA with 22 features and 17 clusters took to run 0.25613190000001396
Neural Netwotk with PCA with 22 features and 17 clusters took to run 0.1150215999999773
PCA with 22 features and 18 clusters took to run 0.28581570000000056
Neural Netwotk with PCA with 22 features and 18 clusters took to run 0.34859830000004877
PCA with 22 features and 19 clusters took to run 0.4139951000000224
Neural Netwotk with PCA with 22 features and 19 clusters took to run 0.03742579999999407
Neural Netwotk with PCA with 23 features took to run 0.03372879999994893
PCA with 23 features and 1 clusters took to run 0.22754509999998618
Neural Netwotk with PCA with 23 features and 1 clusters took to run 0.032457600000043385
PCA with 23 features and 2 clusters took to run 0.07078569999998763
Neural Netwotk with PCA with 23 features and 2 clusters took to run 0.03614570000002004
PCA with 23 features and 3 clusters took to run 0.3862

PCA with 25 features and 11 clusters took to run 0.2102413999999726
Neural Netwotk with PCA with 25 features and 11 clusters took to run 0.09677589999995462
PCA with 25 features and 12 clusters took to run 0.25822410000000673
Neural Netwotk with PCA with 25 features and 12 clusters took to run 0.12689869999996972
PCA with 25 features and 13 clusters took to run 0.27021799999999985
Neural Netwotk with PCA with 25 features and 13 clusters took to run 0.3171389999999974
PCA with 25 features and 14 clusters took to run 0.26160540000000765
Neural Netwotk with PCA with 25 features and 14 clusters took to run 0.9892782000000011
PCA with 25 features and 15 clusters took to run 0.2701220999999805
Neural Netwotk with PCA with 25 features and 15 clusters took to run 0.14451270000000704
PCA with 25 features and 16 clusters took to run 0.299590099999989
Neural Netwotk with PCA with 25 features and 16 clusters took to run 0.1458622000000105
PCA with 25 features and 17 clusters took to run 0.30878309

Neural Netwotk with PCA with 28 features and 5 clusters took to run 0.2687440000000265
PCA with 28 features and 6 clusters took to run 0.15556970000000092
Neural Netwotk with PCA with 28 features and 6 clusters took to run 0.028829900000005182
PCA with 28 features and 7 clusters took to run 0.16344859999998107
Neural Netwotk with PCA with 28 features and 7 clusters took to run 0.16206269999997858
PCA with 28 features and 8 clusters took to run 0.16464289999998982
Neural Netwotk with PCA with 28 features and 8 clusters took to run 0.15686800000003132
PCA with 28 features and 9 clusters took to run 0.19726550000001453
Neural Netwotk with PCA with 28 features and 9 clusters took to run 0.24095219999998108
PCA with 28 features and 10 clusters took to run 0.2092478000000142
Neural Netwotk with PCA with 28 features and 10 clusters took to run 0.11002840000003289
PCA with 28 features and 11 clusters took to run 0.2563923000000159
Neural Netwotk with PCA with 28 features and 11 clusters took t

PCA with 30 features and 19 clusters took to run 0.34562199999999166
Neural Netwotk with PCA with 30 features and 19 clusters took to run 0.03349050000002762
Neural Netwotk with PCA with 31 features took to run 19.987069899999995
PCA with 31 features and 1 clusters took to run 0.04164459999998371
Neural Netwotk with PCA with 31 features and 1 clusters took to run 0.03449820000003001
PCA with 31 features and 2 clusters took to run 0.07634960000001456
Neural Netwotk with PCA with 31 features and 2 clusters took to run 0.03231009999996104
PCA with 31 features and 3 clusters took to run 0.08039949999999862
Neural Netwotk with PCA with 31 features and 3 clusters took to run 0.12953360000000202
PCA with 31 features and 4 clusters took to run 0.12983590000004597
Neural Netwotk with PCA with 31 features and 4 clusters took to run 0.10203289999998333
PCA with 31 features and 5 clusters took to run 0.11933970000001182
Neural Netwotk with PCA with 31 features and 5 clusters took to run 0.24572679

PCA with 33 features and 14 clusters took to run 0.3727761999999757
Neural Netwotk with PCA with 33 features and 14 clusters took to run 1.3086989000000244
PCA with 33 features and 15 clusters took to run 0.34558699999996634
Neural Netwotk with PCA with 33 features and 15 clusters took to run 0.1317081999999914
PCA with 33 features and 16 clusters took to run 0.3383640999999784
Neural Netwotk with PCA with 33 features and 16 clusters took to run 2.0173728000000324
PCA with 33 features and 17 clusters took to run 0.3179969999999912
Neural Netwotk with PCA with 33 features and 17 clusters took to run 0.156559800000025
PCA with 33 features and 18 clusters took to run 0.39095959999997376
Neural Netwotk with PCA with 33 features and 18 clusters took to run 0.9778880000000072
PCA with 33 features and 19 clusters took to run 0.3920355999999856
Neural Netwotk with PCA with 33 features and 19 clusters took to run 0.03399269999999888
Neural Netwotk with PCA with 34 features took to run 10.851080

In [12]:
print(homogeneity_sol)
print(neuralnetwork_sol)

   Algorithm  Components  Clusters  Homogeneity          WCSS      Time
0       Test           0         0     0.000000      0.000000  0.000000
0        PCA           1         1     0.000000   8559.286175  0.513326
0        PCA           1         2     0.194123   1985.593096  0.195220
0        PCA           1         3     0.194880    573.109900  0.301423
0        PCA           1         4     0.222835    339.949151  0.347306
..       ...         ...       ...          ...           ...       ...
0        PCA          34        15     0.309815  13889.329509  0.355639
0        PCA          34        16     0.326566  13323.972123  0.396902
0        PCA          34        17     0.328662  12946.712310  0.353082
0        PCA          34        18     0.308292  12526.905446  0.410136
0        PCA          34        19     0.294683  12275.045683  0.423092

[647 rows x 6 columns]
   Algorithm  Components  Clusters  AccuracyIn  AccuracyOut       Time
0       Base           0         0    0.8

In [13]:
labels = []
for ite in range(1,35):
    t.tic()
    kmeans = KMeans(n_clusters = ite, random_state = 19830526)
    kmeans.fit(corpus)
    labels.append(kmeans.labels_)
    taux = t.tocvalue()
    print("RAW with "+str(ite)+" clusters took to run " + str(taux))
    new_row = pd.DataFrame([["RAW",0,ite,homogeneity_score(label_real, kmeans.labels_),kmeans.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
    homogeneity_sol = pd.concat([homogeneity_sol,new_row])
    t.tic()
    nnBP.fit(pd.get_dummies(kmeans.labels_), dfTrainingY.values.ravel())
    taux = t.tocvalue()
    dfTestY_Pred = nnBP.predict(pd.get_dummies(kmeans.predict(dfTestX)))
    dfTrainingY_Pred = nnBP.predict(pd.get_dummies(kmeans.labels_))
    print("Neural Netwotk after Clustring with "+str(ite)+" clusters took to run " + str(taux))
    new_row = pd.DataFrame([["RAW",0,ite,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
    neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])

RAW with 1 clusters took to run 0.06042539999998553
Neural Netwotk after Clustring with 1 clusters took to run 0.03136729999999943
RAW with 2 clusters took to run 0.09924530000000686
Neural Netwotk after Clustring with 2 clusters took to run 0.030536899999958678
RAW with 3 clusters took to run 0.09585379999998622
Neural Netwotk after Clustring with 3 clusters took to run 0.13368169999995416
RAW with 4 clusters took to run 0.11541399999998703
Neural Netwotk after Clustring with 4 clusters took to run 0.10347670000004427
RAW with 5 clusters took to run 0.1800956999999812
Neural Netwotk after Clustring with 5 clusters took to run 0.31152600000001485
RAW with 6 clusters took to run 0.15682700000002114
Neural Netwotk after Clustring with 6 clusters took to run 0.029360000000053788
RAW with 7 clusters took to run 0.16876089999999522
Neural Netwotk after Clustring with 7 clusters took to run 0.07626440000001367
RAW with 8 clusters took to run 0.174751399999991
Neural Netwotk after Clustring w

In [14]:
#ICA = FastICA(n_components=200,max_iter=1000)
#corpus_ica=ICA.fit_transform(corpus)

In [15]:
labels_ica = []

for iteCom in range(1,34):    
    t.tic()
    ICA = FastICA(n_components=iteCom,max_iter=1000, random_state = 19830526)
    ICA.fit(corpus)        
    corpus_ica=ICA.transform(corpus)        
    t.toc("ICA calibration for "+str(iteCom)+" components took ")
    dfTestXICA = ICA.transform(dfTestX)
    dfTrainingXICA =  ICA.transform(dfTrainingX)
    t.tic()
    nnBP.fit(dfTrainingXICA, dfTrainingY.values.ravel())
    taux = t.tocvalue()
    dfTestY_Pred = nnBP.predict(dfTestXICA)
    dfTrainingY_Pred = nnBP.predict(dfTrainingXICA)
    print("Neural Netwotk with ICA with "+str(iteCom)+" features took to run " + str(taux))
    new_row = pd.DataFrame([["ICA",iteCom,0,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
    neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
    for ite in range(1,20):
        t.tic()
        kmeans_ica = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_ica.fit(corpus_ica)
        labels_ica.append(kmeans_ica.labels_)
        taux = t.tocvalue()
        print("ICA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["ICA",iteCom,ite,homogeneity_score(label_real, kmeans_ica.labels_),kmeans_ica.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])
        t.tic()
        nnBP.fit(pd.get_dummies(kmeans_ica.labels_), dfTrainingY.values.ravel())
        taux = t.tocvalue()
        try:
            dfTestY_Pred = nnBP.predict(pd.get_dummies(kmeans_ica.predict(dfTestXICA)))
            dfTrainingY_Pred = nnBP.predict(pd.get_dummies(kmeans_ica.labels_))
            print("Neural Netwotk with ICA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
            new_row = pd.DataFrame([["ICA",iteCom,ite,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
            neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
        except:
            new_row = pd.DataFrame([["ICA",iteCom,ite,0,0,0]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
            neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 1 components took  0.351988 seconds.
Neural Netwotk with ICA with 1 features took to run 0.029444000000012238
ICA with 1 features and 1 clusters took to run 0.020522299999925053
Neural Netwotk with ICA with 1 features and 1 clusters took to run 0.0362168000000338
ICA with 1 features and 2 clusters took to run 0.06569409999997333
Neural Netwotk with ICA with 1 features and 2 clusters took to run 0.03727090000006683
ICA with 1 features and 3 clusters took to run 0.05590719999997873
Neural Netwotk with ICA with 1 features and 3 clusters took to run 0.11551880000001802
ICA with 1 features and 4 clusters took to run 0.0646635000000515
Neural Netwotk with ICA with 1 features and 4 clusters took to run 0.13399409999999534
ICA with 1 features and 5 clusters took to run 0.08319059999996625
Neural Netwotk with ICA with 1 features and 5 clusters took to run 0.10333769999999731
ICA with 1 features and 6 clusters took to run 0.06890699999996741
Neural Netwotk with ICA with 1 fea

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA with 2 features and 2 clusters took to run 0.04022799999995641
Neural Netwotk with ICA with 2 features and 2 clusters took to run 0.03134659999989253
ICA with 2 features and 3 clusters took to run 0.04644369999994069
Neural Netwotk with ICA with 2 features and 3 clusters took to run 0.08215210000003026
ICA with 2 features and 4 clusters took to run 0.05357010000000173
Neural Netwotk with ICA with 2 features and 4 clusters took to run 0.09303339999996751
ICA with 2 features and 5 clusters took to run 0.081512900000007
Neural Netwotk with ICA with 2 features and 5 clusters took to run 0.19105930000000626
ICA with 2 features and 6 clusters took to run 0.07158220000007987
Neural Netwotk with ICA with 2 features and 6 clusters took to run 0.03251389999991261
ICA with 2 features and 7 clusters took to run 0.09187659999997777
Neural Netwotk with ICA with 2 features and 7 clusters took to run 0.5799438999999893
ICA with 2 features and 8 clusters took to run 0.09472909999999501
Neural Netwo

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


...

ICA calibration for 31 components took  6.604897 seconds.
Neural Netwotk with ICA with 31 features took to run 16.34619230000021
ICA with 31 features and 1 clusters took to run 0.24780390000000807
Neural Netwotk with ICA with 31 features and 1 clusters took to run 0.0420145999999022
ICA with 31 features and 2 clusters took to run 0.6597506000000521
Neural Netwotk with ICA with 31 features and 2 clusters took to run 0.04153629999996156
ICA with 31 features and 3 clusters took to run 0.9460636999999679
Neural Netwotk with ICA with 31 features and 3 clusters took to run 0.12259910000011587
ICA with 31 features and 4 clusters took to run 0.9706360000000132
Neural Netwotk with ICA with 31 features and 4 clusters took to run 0.10063839999997981
ICA with 31 features and 5 clusters took to run 0.2712460000000192
Neural Netwotk with ICA with 31 features and 5 clusters took to run 1.0344253999999182
ICA with 31 features and 6 clusters took to run 1.3145264000002044
Neural Netwotk with ICA with 3

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 32 components took  2.836051 seconds.
Neural Netwotk with ICA with 32 features took to run 0.07630719999997382
ICA with 32 features and 1 clusters took to run 0.20640949999983604
Neural Netwotk with ICA with 32 features and 1 clusters took to run 0.050249699999994846
ICA with 32 features and 2 clusters took to run 0.6063380000000507
Neural Netwotk with ICA with 32 features and 2 clusters took to run 0.04365890000008221
ICA with 32 features and 3 clusters took to run 0.9962901999999758
Neural Netwotk with ICA with 32 features and 3 clusters took to run 0.05359889999999723
ICA with 32 features and 4 clusters took to run 0.3753153000000111
Neural Netwotk with ICA with 32 features and 4 clusters took to run 0.22393130000000383
ICA with 32 features and 5 clusters took to run 0.3501369999999042
Neural Netwotk with ICA with 32 features and 5 clusters took to run 0.657962100000077
ICA with 32 features and 6 clusters took to run 1.2266869000000042
Neural Netwotk with ICA wit

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\decomposition\_fastica.py:494: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


ICA calibration for 33 components took  4.267855 seconds.
Neural Netwotk with ICA with 33 features took to run 27.462623699999995
ICA with 33 features and 1 clusters took to run 0.2584369000001061
Neural Netwotk with ICA with 33 features and 1 clusters took to run 0.06159579999984999
ICA with 33 features and 2 clusters took to run 0.37881579999998394
Neural Netwotk with ICA with 33 features and 2 clusters took to run 0.06818360000011126
ICA with 33 features and 3 clusters took to run 0.38819899999998597
Neural Netwotk with ICA with 33 features and 3 clusters took to run 0.07153180000000248
ICA with 33 features and 4 clusters took to run 0.30589839999993274
Neural Netwotk with ICA with 33 features and 4 clusters took to run 0.0851868000002014
ICA with 33 features and 5 clusters took to run 0.28144330000009177
ICA with 33 features and 6 clusters took to run 1.442283100000168
Neural Netwotk with ICA with 33 features and 6 clusters took to run 0.06330229999980475
ICA with 33 features and 7

In [16]:
print(corpus.shape)
print(len(label_real))

(16000, 35)
16000


In [17]:
#RCA = random_projection.SparseRandomProjection()
#corpus_rca = RCA.fit_transform(corpus)

In [18]:
labels_rca = []
for iteCom in range(1,35):
    t.tic()
    RCA = random_projection.SparseRandomProjection(n_components=iteCom, random_state = 19830526)
    RCA.fit(corpus)
    corpus_rca = RCA.transform(corpus)
    t.toc("RCA calibration for "+str(iteCom)+" components took ")
    dfTestXRCA = RCA.transform(dfTestX)
    dfTrainingXRCA =  RCA.transform(dfTrainingX)
    t.tic()
    nnBP.fit(dfTrainingXRCA, dfTrainingY.values.ravel())
    taux = t.tocvalue()
    dfTestY_Pred = nnBP.predict(dfTestXRCA)
    dfTrainingY_Pred = nnBP.predict(dfTrainingXRCA)
    print("Neural Netwotk with RCA with "+str(iteCom)+" features took to run " + str(taux))
    new_row = pd.DataFrame([["RCA",iteCom,0,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
    neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
    for ite in range(1,20):
        t.tic()
        kmeans_rca = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_rca.fit(corpus_rca)
        labels_rca.append(kmeans_rca.labels_)
        taux = t.tocvalue()
        print("RCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["RCA",iteCom,ite,homogeneity_score(label_real, kmeans_rca.labels_),kmeans_rca.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])
        t.tic()
        nnBP.fit(pd.get_dummies(kmeans_rca.labels_), dfTrainingY.values.ravel())
        taux = t.tocvalue()
        try:
            dfTestY_Pred = nnBP.predict(pd.get_dummies(kmeans_rca.predict(dfTestXRCA)))
            dfTrainingY_Pred = nnBP.predict(pd.get_dummies(kmeans_rca.labels_))
            print("Neural Netwotk with RCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
            new_row = pd.DataFrame([["RCA",iteCom,ite,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
            neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
        except:
            new_row = pd.DataFrame([["RCA",iteCom,ite,0,0,0]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
            neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])           

RCA calibration for 1 components took  0.042648 seconds.
Neural Netwotk with RCA with 1 features took to run 0.04688880000003337
RCA with 1 features and 1 clusters took to run 0.021318000000064785
Neural Netwotk with RCA with 1 features and 1 clusters took to run 0.03690010000013899
RCA with 1 features and 2 clusters took to run 0.03760640000018611
Neural Netwotk with RCA with 1 features and 2 clusters took to run 0.0383911999999782
RCA with 1 features and 3 clusters took to run 0.04195979999985866
Neural Netwotk with RCA with 1 features and 3 clusters took to run 0.10747250000008535
RCA with 1 features and 4 clusters took to run 0.047709800000120595
Neural Netwotk with RCA with 1 features and 4 clusters took to run 0.07443890000013198
RCA with 1 features and 5 clusters took to run 0.06449410000004718
Neural Netwotk with RCA with 1 features and 5 clusters took to run 0.378623399999924
RCA with 1 features and 6 clusters took to run 0.17195250000008855
Neural Netwotk with RCA with 1 feat

Neural Netwotk with RCA with 3 features and 14 clusters took to run 0.3320160999999189
RCA with 3 features and 15 clusters took to run 0.3422218000000612
Neural Netwotk with RCA with 3 features and 15 clusters took to run 0.18393289999994522
RCA with 3 features and 16 clusters took to run 0.37992289999988316
Neural Netwotk with RCA with 3 features and 16 clusters took to run 0.2695151000000351
RCA with 3 features and 17 clusters took to run 0.896105900000066
Neural Netwotk with RCA with 3 features and 17 clusters took to run 0.9600874000000204
RCA with 3 features and 18 clusters took to run 1.2364966000000095
Neural Netwotk with RCA with 3 features and 18 clusters took to run 0.3031181999999717
RCA with 3 features and 19 clusters took to run 1.1503402999999253
Neural Netwotk with RCA with 3 features and 19 clusters took to run 0.5206336999999621
RCA calibration for 4 components took  0.017061 seconds.
Neural Netwotk with RCA with 4 features took to run 1.296695199999931
RCA with 4 feat

Neural Netwotk with RCA with 6 features and 8 clusters took to run 0.1668225999999322
RCA with 6 features and 9 clusters took to run 1.717831500000102
Neural Netwotk with RCA with 6 features and 9 clusters took to run 0.1631657999998879
RCA with 6 features and 10 clusters took to run 1.435288700000001
Neural Netwotk with RCA with 6 features and 10 clusters took to run 0.3192618999999013
RCA with 6 features and 11 clusters took to run 1.5319260999999642
Neural Netwotk with RCA with 6 features and 11 clusters took to run 0.1568888000001607
RCA with 6 features and 12 clusters took to run 0.5311788999999862
Neural Netwotk with RCA with 6 features and 12 clusters took to run 0.34389930000020286
RCA with 6 features and 13 clusters took to run 0.4419834999998784
Neural Netwotk with RCA with 6 features and 13 clusters took to run 0.05039139999985309
RCA with 6 features and 14 clusters took to run 1.839309700000058
Neural Netwotk with RCA with 6 features and 14 clusters took to run 0.2932774000

RCA with 9 features and 3 clusters took to run 1.033701899999869
Neural Netwotk with RCA with 9 features and 3 clusters took to run 0.11894069999993917
RCA with 9 features and 4 clusters took to run 1.1513453000000027
Neural Netwotk with RCA with 9 features and 4 clusters took to run 0.08394130000010591
RCA with 9 features and 5 clusters took to run 0.8728092000001197
Neural Netwotk with RCA with 9 features and 5 clusters took to run 0.11534469999992325
RCA with 9 features and 6 clusters took to run 0.33922770000003766
Neural Netwotk with RCA with 9 features and 6 clusters took to run 0.04245589999982258
RCA with 9 features and 7 clusters took to run 0.35715659999982563
Neural Netwotk with RCA with 9 features and 7 clusters took to run 0.18349929999999404
RCA with 9 features and 8 clusters took to run 1.004367499999944
Neural Netwotk with RCA with 9 features and 8 clusters took to run 0.17245589999993172
RCA with 9 features and 9 clusters took to run 1.5279163000000153
Neural Netwotk w

Neural Netwotk with RCA with 11 features and 16 clusters took to run 1.088832899999943
RCA with 11 features and 17 clusters took to run 0.6242297000001145
Neural Netwotk with RCA with 11 features and 17 clusters took to run 0.44571330000007947
RCA with 11 features and 18 clusters took to run 2.0894591999999648
Neural Netwotk with RCA with 11 features and 18 clusters took to run 0.7498674999999366
RCA with 11 features and 19 clusters took to run 2.160415400000147
Neural Netwotk with RCA with 11 features and 19 clusters took to run 0.04146020000007411
RCA calibration for 12 components took  0.020736 seconds.
Neural Netwotk with RCA with 12 features took to run 2.0762555000001157
RCA with 12 features and 1 clusters took to run 0.15271059999986392
Neural Netwotk with RCA with 12 features and 1 clusters took to run 0.0862741000000824
RCA with 12 features and 2 clusters took to run 0.5861124000000473
Neural Netwotk with RCA with 12 features and 2 clusters took to run 0.06197950000000674
RCA 

...

RCA with 22 features and 11 clusters took to run 0.3438277999998718
Neural Netwotk with RCA with 22 features and 11 clusters took to run 0.08402559999990444
RCA with 22 features and 12 clusters took to run 0.279246300000068
Neural Netwotk with RCA with 22 features and 12 clusters took to run 0.22950189999983195
RCA with 22 features and 13 clusters took to run 0.3069514999999683
Neural Netwotk with RCA with 22 features and 13 clusters took to run 0.39897610000002715
RCA with 22 features and 14 clusters took to run 0.3089343999999983
Neural Netwotk with RCA with 22 features and 14 clusters took to run 0.2770246000000043
RCA with 22 features and 15 clusters took to run 0.3033999000001586
Neural Netwotk with RCA with 22 features and 15 clusters took to run 0.2877332000000479
RCA with 22 features and 16 clusters took to run 0.31251879999990706
Neural Netwotk with RCA with 22 features and 16 clusters took to run 0.2479576000000634
RCA with 22 features and 17 clusters took to run 0.4665405999

RCA with 25 features and 5 clusters took to run 0.3481410000001688
Neural Netwotk with RCA with 25 features and 5 clusters took to run 0.13649259999988317
RCA with 25 features and 6 clusters took to run 0.40140400000018417
Neural Netwotk with RCA with 25 features and 6 clusters took to run 0.04057209999996303
RCA with 25 features and 7 clusters took to run 0.4705483999998705
Neural Netwotk with RCA with 25 features and 7 clusters took to run 0.1346616000000722
RCA with 25 features and 8 clusters took to run 1.0508418000001711
Neural Netwotk with RCA with 25 features and 8 clusters took to run 0.12840200000005098
RCA with 25 features and 9 clusters took to run 1.697292999999945
Neural Netwotk with RCA with 25 features and 9 clusters took to run 0.41077919999997903
RCA with 25 features and 10 clusters took to run 1.6003719000000274
Neural Netwotk with RCA with 25 features and 10 clusters took to run 0.16897560000006706
RCA with 25 features and 11 clusters took to run 0.7982013999999253
N

Neural Netwotk with RCA with 27 features and 18 clusters took to run 1.5421708999997463
RCA with 27 features and 19 clusters took to run 2.842981900000268
Neural Netwotk with RCA with 27 features and 19 clusters took to run 1.2161041999997906
RCA calibration for 28 components took  0.071988 seconds.
Neural Netwotk with RCA with 28 features took to run 59.248855000000276
RCA with 28 features and 1 clusters took to run 0.3832941999999093
Neural Netwotk with RCA with 28 features and 1 clusters took to run 0.0928367999999864
RCA with 28 features and 2 clusters took to run 0.9267500000000837
Neural Netwotk with RCA with 28 features and 2 clusters took to run 0.07577130000026955
RCA with 28 features and 3 clusters took to run 1.0547301000001426
Neural Netwotk with RCA with 28 features and 3 clusters took to run 0.7717674000000443
RCA with 28 features and 4 clusters took to run 0.8860709000000497
Neural Netwotk with RCA with 28 features and 4 clusters took to run 0.426763899999969
RCA with 28

RCA with 30 features and 12 clusters took to run 1.8488695000000916
Neural Netwotk with RCA with 30 features and 12 clusters took to run 0.5973438999999416
RCA with 30 features and 13 clusters took to run 1.7469207000003735
Neural Netwotk with RCA with 30 features and 13 clusters took to run 0.6722693000001527
RCA with 30 features and 14 clusters took to run 1.63573389999965
Neural Netwotk with RCA with 30 features and 14 clusters took to run 1.1184496999999283
RCA with 30 features and 15 clusters took to run 1.7008626999995613
Neural Netwotk with RCA with 30 features and 15 clusters took to run 0.35478710000006686
RCA with 30 features and 16 clusters took to run 1.5893444999996973
Neural Netwotk with RCA with 30 features and 16 clusters took to run 0.6564818000001651
RCA with 30 features and 17 clusters took to run 1.881796199999826
Neural Netwotk with RCA with 30 features and 17 clusters took to run 2.2192348999997193
RCA with 30 features and 18 clusters took to run 2.19572560000006


Neural Netwotk with RCA with 33 features and 5 clusters took to run 0.28907059999983176
RCA with 33 features and 6 clusters took to run 0.7771487999998499
Neural Netwotk with RCA with 33 features and 6 clusters took to run 0.03456000000005588
RCA with 33 features and 7 clusters took to run 0.5967258000000584
Neural Netwotk with RCA with 33 features and 7 clusters took to run 0.27823199999966164
RCA with 33 features and 8 clusters took to run 4.675124500000038
Neural Netwotk with RCA with 33 features and 8 clusters took to run 0.20126630000004297
RCA with 33 features and 9 clusters took to run 5.350510499999928
Neural Netwotk with RCA with 33 features and 9 clusters took to run 0.4419781999999941
RCA with 33 features and 10 clusters took to run 4.659487800000079
Neural Netwotk with RCA with 33 features and 10 clusters took to run 0.17431400000032227
RCA with 33 features and 11 clusters took to run 0.9481234999998378
Neural Netwotk with RCA with 33 features and 11 clusters took to run 0.

In [19]:
labels_lda = []
   
iteCom = 1
t.tic()
LDA = LinearDiscriminantAnalysis(n_components=iteCom)
LDA.fit(corpus, label_real)
corpus_lda = LDA.transform(corpus)
t.toc("LDA calibration for "+str(iteCom)+" components took ")
dfTestXLDA = LDA.transform(dfTestX)
dfTrainingXLDA =  LDA.transform(dfTrainingX)
t.tic()
nnBP.fit(dfTrainingXLDA, dfTrainingY.values.ravel())
taux = t.tocvalue()
dfTestY_Pred = nnBP.predict(dfTestXLDA)
dfTrainingY_Pred = nnBP.predict(dfTrainingXLDA)
print("Neural Netwotk with LDA with "+str(iteCom)+" features took to run " + str(taux))
new_row = pd.DataFrame([["LDA",iteCom,0,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])
for ite in range(1,20):
        t.tic()
        kmeans_lda = KMeans(n_clusters = ite, random_state = 19830526)
        kmeans_lda.fit(corpus_lda)
        labels_lda.append(kmeans_lda.labels_)    
        taux = t.tocvalue()
        print("LCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["LDA",iteCom,ite,homogeneity_score(label_real, kmeans_lda.labels_),kmeans_lda.inertia_,taux]],columns=["Algorithm","Components","Clusters","Homogeneity","WCSS","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])
        t.tic()
        nnBP.fit(pd.get_dummies(kmeans_lda.labels_), dfTrainingY.values.ravel())
        taux = t.tocvalue()
        dfTestY_Pred = nnBP.predict(pd.get_dummies(kmeans_lda.predict(dfTestXLDA)))
        dfTrainingY_Pred = nnBP.predict(pd.get_dummies(kmeans_lda.labels_))
        print("Neural Netwotk with LDA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["LDA",iteCom,ite,accuracy_score(dfTrainingY, dfTrainingY_Pred),accuracy_score(dfTestY, dfTestY_Pred),taux]],columns=["Algorithm","Components","Clusters","AccuracyIn","AccuracyOut","Time"])
        neuralnetwork_sol = pd.concat([neuralnetwork_sol,new_row])

LDA calibration for 1 components took  0.202925 seconds.
Neural Netwotk with LDA with 1 features took to run 0.07840189999978975
LCA with 1 features and 1 clusters took to run 0.05324369999971168
Neural Netwotk with LDA with 1 features and 1 clusters took to run 0.036998099999891565
LCA with 1 features and 2 clusters took to run 0.2774165000000721
Neural Netwotk with LDA with 1 features and 2 clusters took to run 0.05625550000013391
LCA with 1 features and 3 clusters took to run 0.12888109999994413
Neural Netwotk with LDA with 1 features and 3 clusters took to run 0.1610411000001477
LCA with 1 features and 4 clusters took to run 0.14962980000018433
Neural Netwotk with LDA with 1 features and 4 clusters took to run 0.08991789999981847
LCA with 1 features and 5 clusters took to run 0.23222740000028352
Neural Netwotk with LDA with 1 features and 5 clusters took to run 0.10295730000007097
LCA with 1 features and 6 clusters took to run 0.2191063000000213
Neural Netwotk with LDA with 1 featu

In [20]:
df = pd.DataFrame(label_real, index=corpus.index ,columns =['Real'])

In [21]:
homogeneity_sol.to_csv("IncomeClusteringResultsKMeans.csv")
neuralnetwork_sol.to_csv("IncomeClusteringResultsNeuralNetwork.csv")

In [26]:
df_ica = pd.DataFrame(labels_ica, columns=corpus.index).T
df_pca = pd.DataFrame(labels_pca, columns=corpus.index).T
df_rca = pd.DataFrame(labels_rca, columns=corpus.index).T
df_lda = pd.DataFrame(labels_lda, columns=corpus.index).T
df_raw = pd.DataFrame(labels, columns=corpus.index).T 
df.to_csv("actuals_Income.csv")
df_ica.to_csv("ica_Income.csv")
df_pca.to_csv("pca_Income.csv")
df_rca.to_csv("rca_Income.csv")
df_lda.to_csv("lda_Income.csv")
df_raw.to_csv("raw_Income.csv")

In [25]:
len(labels_ica[1])

16000